# 置信区间

整体通过检验Pass the inspection wholely.
置信度Degree of confidence
置信区间 Range of confidence

预测区间，预测区间是观测值的置信区间，包括误差估计。([预测区间比置信区间宽](https://www.zhihu.com/question/263893098/answer/676012258))

预测区间估计(prediction interval estimate)：利用估计的回归方程，对于自变量 x 的一个给定值 x0 ，求出因变量 y 的一个个别值的估计区间。变量的估计叫预测区间，预测区间反映了单个数值的不确定性；

置信区间估计(confidence interval estimate)：利用估计的回归方程，对于自变量 x 的一个给定值 x0 ，求出因变量 y 的平均值的估计区间。参数的估计叫置信区间，置信区间反映了预测均值的不确定性。

![](https://github.com/openjw/open/blob/master/MachineLearning/img/Range_of_confidence.png?raw=true)
## 贝叶斯线性回归
predict(X[, return_std]) 可以返回方差（有了均值和方差就可以计算预测区间了）



## 其它
[回归预测值预测区间_第四十讲 R-线性回归：预测模型及可信区间](https://blog.csdn.net/weixin_30011833/article/details/112624965)
[回归预测值预测区间_【Origin】通过线性拟合添加置信区间](https://blog.csdn.net/weixin_39894778/article/details/112624975)
[confidence interval](https://github.com/coolcalbeans/PEPredictor)

[使用分位数回归预测目标的取值范围](https://www.jianshu.com/p/073bc0b0bbfa)

[如何近似计算回归方程的预测区间](https://blog.csdn.net/dylanren/article/details/105841919)

[如何理解 95% 置信区间？](https://www.zhihu.com/question/26419030/answer/70589735)
## MAPIE
https://github.com/simai-ml/MAPIE

https://mapie.readthedocs.io/en/latest/tutorial.html

## statsmodels伪代码
> [statsmodels模块提供置信区间](https://github.com/statsmodels/statsmodels)

1. 方法一？

```

import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

n = 100

x = np.linspace(0, 10, n)
e = np.random.normal(size=n)
y = 1 + 0.5*x + 2*e
X = sm.add_constant(x)

re = sm.OLS(y, X).fit()
print(re.summary())

prstd, iv_l, iv_u = wls_prediction_std(re)
# iv_l和iv_u是上下置信区间
```

2. 方法二？

```
from statsmodels.stats.outliers_influence import summary_table

st, data, ss2 = summary_table(re, alpha=0.05)

fittedvalues = data[:, 2]
predict_mean_se  = data[:, 3]
predict_mean_ci_low, predict_mean_ci_upp = data[:, 4:6].T # 上下区间
predict_ci_low, predict_ci_upp = data[:, 6:8].T # 上下区间

# Check we got the right things
print np.max(np.abs(re.fittedvalues - fittedvalues))
print np.max(np.abs(iv_l - predict_ci_low))
print np.max(np.abs(iv_u - predict_ci_upp))

plt.plot(x, y, 'o')
plt.plot(x, fittedvalues, '-', lw=2)
plt.plot(x, predict_ci_low, 'r--', lw=2)
plt.plot(x, predict_ci_upp, 'r--', lw=2)
plt.plot(x, predict_mean_ci_low, 'r--', lw=2)
plt.plot(x, predict_mean_ci_upp, 'r--', lw=2)
plt.show()
```

3. 方法三？

```
predictions = re.get_prediction(out_of_sample_df)
predictions.summary_frame(alpha=0.05) # 通过修改“alpha”参数，可以更改置信区间和预测区间的显著性级别。

# 下面是一个函数，使用此方法获取模型、新数据和任意分位数：

def ols_quantile(m, X, q):
  # m: OLS model.
  # X: X matrix.
  # q: Quantile.
  #
  # Set alpha based on q.
  a = q * 2
  if q > 0.5:
    a = 2 * (1 - q)
  predictions = m.get_prediction(X)
  frame = predictions.summary_frame(alpha=a)
  if q > 0.5:
    return frame.obs_ci_upper
  return frame.obs_ci_lower
```

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model

In [2]:
class LRPI:
    def __init__(self, normalize=False, n_jobs=1, t_value = 2.13144955):
        self.normalize = normalize
        self.n_jobs = n_jobs
        self.LR = linear_model.LinearRegression(normalize=self.normalize, n_jobs= self.n_jobs)
        self.t_value = t_value

    def fit(self, X_train, y_train):
        self.X_train = pd.DataFrame(X_train.values)
        self.y_train = pd.DataFrame(y_train.values)

        self.LR.fit(self.X_train, self.y_train)
        X_train_fit = self.LR.predict(self.X_train)
        self.MSE = np.power(self.y_train.subtract(X_train_fit), 2).sum(axis=0) / (self.X_train.shape[0] - self.X_train.shape[1] - 1)
        self.X_train.loc[:, 'const_one'] = 1
        self.XTX_inv = np.linalg.inv(np.dot(np.transpose(self.X_train.values) , self.X_train.values))

    def predict(self, X_test):
        self.X_test = pd.DataFrame(X_test.values)
        self.pred = self.LR.predict(self.X_test)
        self.X_test.loc[: , 'const_one'] =1
        SE = [np.dot(np.transpose(self.X_test.values[i]) , np.dot(self.XTX_inv, self.X_test.values[i]) ) for i in range(len(self.X_test)) ]
        results = pd.DataFrame(self.pred , columns=['Pred'])
        # 预测区间
        results.loc[:,"lower"] = results['Pred'].subtract((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        results.loc[:,"upper"] = results['Pred'].add((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)

        return results

In [3]:
data = pd.DataFrame(datasets.load_diabetes().data)
target = pd.DataFrame(datasets.load_diabetes().target)

In [4]:
X_train = data.iloc[:-30 ]
y_train = target.iloc[: -30 ]

X_test = data.iloc[-30: ]
y_test = target.iloc[-30: ]



In [5]:
model = LRPI()
model.fit(X_train, y_train)



In [6]:
results = model.predict(X_test)
results.head(10)


,Pred,lower,upper
0,234.040695,115.313069,352.768320
1,122.922466,4.996653,240.848279
2,166.354331,47.642949,285.065713
3,174.392229,56.049506,292.734952
4,226.948514,107.843118,346.053910
5,151.821902,33.002894,270.640910
6,100.943585,-17.585884,219.473054
7,83.089561,-35.107119,201.286241
8,143.129867,24.313062,261.946671
9,192.678237,74.918278,310.438195
